# Reading the board power status
We have now seen a couple of cases where one of the power indicator LEDs (other than VADJ_FMC, which is always off for us) is off. This is presumably a board defect. It has so far been seen on DAC_AVCC and ADC_AVCC. In these cases, the symptom was that the DAC/ADC side of the RFDC would either not lock to the reference clock, or would behave in a way that suggests that the clock is bad (DAC playing signals at ~0.5x the expected speed).

The RFSoC boards have voltage+current monitoring on the board power supplies. The easiest way to read them back is by installing lm-sensors (`sudo apt install lm-sensors`) and running `sensors`, but this is harder if the board's not connected to the Internet. Here's some code that does the same thing, but without installing anything extra.

It's useful to run this after power-cycling the board (before any firmware is loaded) and after loading the standard QICK firmware by initializing `QickSoc()`.

This code copies heavily from `pynq.pmbus`.

In [1]:
# a rough equivalent of installing lm-sensors and running the "sensors" command
# based on pynq.pmbus, but just printing all the sensors instead of enumerating them

#   Copyright (c) 2021, Xilinx, Inc.
#   SPDX-License-Identifier: BSD-3-Clause

import cffi

_c_header = R"""
extern const char *libsensors_version;

typedef struct sensors_bus_id {
 short type;
 short nr;
} sensors_bus_id;

typedef struct sensors_chip_name {
 char *prefix;
 sensors_bus_id bus;
 int addr;
 char *path;
} sensors_chip_name;

int sensors_init(FILE *input);
void sensors_cleanup(void);
int sensors_parse_chip_name(const char *orig_name, sensors_chip_name *res);
void sensors_free_chip_name(sensors_chip_name *chip);
int sensors_snprintf_chip_name(char *str, size_t size,
          const sensors_chip_name *chip);
const char *sensors_get_adapter_name(const sensors_bus_id *bus);

typedef struct sensors_feature sensors_feature;
char *sensors_get_label(const sensors_chip_name *name,
   const sensors_feature *feature);

int sensors_get_value(const sensors_chip_name *name, int subfeat_nr,
        double *value);

int sensors_set_value(const sensors_chip_name *name, int subfeat_nr,
        double value);

int sensors_do_chip_sets(const sensors_chip_name *name);

const sensors_chip_name *sensors_get_detected_chips(const sensors_chip_name
          *match, int *nr);

typedef enum sensors_feature_type {
 SENSORS_FEATURE_IN = 0x00,
 SENSORS_FEATURE_FAN = 0x01,
 SENSORS_FEATURE_TEMP = 0x02,
 SENSORS_FEATURE_POWER = 0x03,
 SENSORS_FEATURE_ENERGY = 0x04,
 SENSORS_FEATURE_CURR = 0x05,
 SENSORS_FEATURE_HUMIDITY = 0x06,
 SENSORS_FEATURE_MAX_MAIN,
 SENSORS_FEATURE_VID = 0x10,
 SENSORS_FEATURE_INTRUSION = 0x11,
 SENSORS_FEATURE_MAX_OTHER,
 SENSORS_FEATURE_BEEP_ENABLE = 0x18,
 SENSORS_FEATURE_MAX,
 SENSORS_FEATURE_UNKNOWN = 0x7fffffff,
} sensors_feature_type;

typedef enum sensors_subfeature_type {
 SENSORS_SUBFEATURE_IN_INPUT = 0,
 SENSORS_SUBFEATURE_IN_MIN,
 SENSORS_SUBFEATURE_IN_MAX,
 SENSORS_SUBFEATURE_IN_LCRIT,
 SENSORS_SUBFEATURE_IN_CRIT,
 SENSORS_SUBFEATURE_IN_AVERAGE,
 SENSORS_SUBFEATURE_IN_LOWEST,
 SENSORS_SUBFEATURE_IN_HIGHEST,
 SENSORS_SUBFEATURE_IN_ALARM = 0x80,
 SENSORS_SUBFEATURE_IN_MIN_ALARM,
 SENSORS_SUBFEATURE_IN_MAX_ALARM,
 SENSORS_SUBFEATURE_IN_BEEP,
 SENSORS_SUBFEATURE_IN_LCRIT_ALARM,
 SENSORS_SUBFEATURE_IN_CRIT_ALARM,

 SENSORS_SUBFEATURE_FAN_INPUT = 0x100,
 SENSORS_SUBFEATURE_FAN_MIN,
 SENSORS_SUBFEATURE_FAN_MAX,
 SENSORS_SUBFEATURE_FAN_ALARM = 0x180,
 SENSORS_SUBFEATURE_FAN_FAULT,
 SENSORS_SUBFEATURE_FAN_DIV,
 SENSORS_SUBFEATURE_FAN_BEEP,
 SENSORS_SUBFEATURE_FAN_PULSES,
 SENSORS_SUBFEATURE_FAN_MIN_ALARM,
 SENSORS_SUBFEATURE_FAN_MAX_ALARM,

 SENSORS_SUBFEATURE_TEMP_INPUT = 0x200,
 SENSORS_SUBFEATURE_TEMP_MAX,
 SENSORS_SUBFEATURE_TEMP_MAX_HYST,
 SENSORS_SUBFEATURE_TEMP_MIN,
 SENSORS_SUBFEATURE_TEMP_CRIT,
 SENSORS_SUBFEATURE_TEMP_CRIT_HYST,
 SENSORS_SUBFEATURE_TEMP_LCRIT,
 SENSORS_SUBFEATURE_TEMP_EMERGENCY,
 SENSORS_SUBFEATURE_TEMP_EMERGENCY_HYST,
 SENSORS_SUBFEATURE_TEMP_LOWEST,
 SENSORS_SUBFEATURE_TEMP_HIGHEST,
 SENSORS_SUBFEATURE_TEMP_MIN_HYST,
 SENSORS_SUBFEATURE_TEMP_LCRIT_HYST,
 SENSORS_SUBFEATURE_TEMP_ALARM = 0x280,
 SENSORS_SUBFEATURE_TEMP_MAX_ALARM,
 SENSORS_SUBFEATURE_TEMP_MIN_ALARM,
 SENSORS_SUBFEATURE_TEMP_CRIT_ALARM,
 SENSORS_SUBFEATURE_TEMP_FAULT,
 SENSORS_SUBFEATURE_TEMP_TYPE,
 SENSORS_SUBFEATURE_TEMP_OFFSET,
 SENSORS_SUBFEATURE_TEMP_BEEP,
 SENSORS_SUBFEATURE_TEMP_EMERGENCY_ALARM,
 SENSORS_SUBFEATURE_TEMP_LCRIT_ALARM,

 SENSORS_SUBFEATURE_POWER_AVERAGE = 0x300,
 SENSORS_SUBFEATURE_POWER_AVERAGE_HIGHEST,
 SENSORS_SUBFEATURE_POWER_AVERAGE_LOWEST,
 SENSORS_SUBFEATURE_POWER_INPUT,
 SENSORS_SUBFEATURE_POWER_INPUT_HIGHEST,
 SENSORS_SUBFEATURE_POWER_INPUT_LOWEST,
 SENSORS_SUBFEATURE_POWER_CAP,
 SENSORS_SUBFEATURE_POWER_CAP_HYST,
 SENSORS_SUBFEATURE_POWER_MAX,
 SENSORS_SUBFEATURE_POWER_CRIT,
 SENSORS_SUBFEATURE_POWER_AVERAGE_INTERVAL = 0x380,
 SENSORS_SUBFEATURE_POWER_ALARM,
 SENSORS_SUBFEATURE_POWER_CAP_ALARM,
 SENSORS_SUBFEATURE_POWER_MAX_ALARM,
 SENSORS_SUBFEATURE_POWER_CRIT_ALARM,

 SENSORS_SUBFEATURE_ENERGY_INPUT = 0x400,

 SENSORS_SUBFEATURE_CURR_INPUT = 0x500,
 SENSORS_SUBFEATURE_CURR_MIN,
 SENSORS_SUBFEATURE_CURR_MAX,
 SENSORS_SUBFEATURE_CURR_LCRIT,
 SENSORS_SUBFEATURE_CURR_CRIT,
 SENSORS_SUBFEATURE_CURR_AVERAGE,
 SENSORS_SUBFEATURE_CURR_LOWEST,
 SENSORS_SUBFEATURE_CURR_HIGHEST,
 SENSORS_SUBFEATURE_CURR_ALARM = 0x580,
 SENSORS_SUBFEATURE_CURR_MIN_ALARM,
 SENSORS_SUBFEATURE_CURR_MAX_ALARM,
 SENSORS_SUBFEATURE_CURR_BEEP,
 SENSORS_SUBFEATURE_CURR_LCRIT_ALARM,
 SENSORS_SUBFEATURE_CURR_CRIT_ALARM,

 SENSORS_SUBFEATURE_HUMIDITY_INPUT = 0x600,

 SENSORS_SUBFEATURE_VID = 0x1000,

 SENSORS_SUBFEATURE_INTRUSION_ALARM = 0x1100,
 SENSORS_SUBFEATURE_INTRUSION_BEEP,

 SENSORS_SUBFEATURE_BEEP_ENABLE = 0x1800,

 SENSORS_SUBFEATURE_UNKNOWN = 0x7fffffff,
} sensors_subfeature_type;


struct sensors_feature {
 char *name;
 int number;
 sensors_feature_type type;

 int first_subfeature;
 int padding1;
};

typedef struct sensors_subfeature {
 char *name;
 int number;
 sensors_subfeature_type type;
 int mapping;
 unsigned int flags;
} sensors_subfeature;

const sensors_feature *
sensors_get_features(const sensors_chip_name *name, int *nr);

const sensors_subfeature *
sensors_get_all_subfeatures(const sensors_chip_name *name,
       const sensors_feature *feature, int *nr);

const sensors_subfeature *
sensors_get_subfeature(const sensors_chip_name *name,
         const sensors_feature *feature,
         sensors_subfeature_type type);
"""

_ffi = cffi.FFI()

try:
    _ffi.cdef(_c_header)
    _lib = _ffi.dlopen("libsensors.so")
except Exception as e:
    _lib = None

def print_sensors(target_chip=None, config_file=None):
    if _lib is None:
        warnings.warn("Could not initialise libsensors library")
        return {}

    _lib.sensors_cleanup()
    if config_file:
        with open(config_file, "r") as handle:
            _lib.sensors_init(handle)
    else:
        _lib.sensors_init(_ffi.NULL)

    chip_nr = _ffi.new("int [1]")
    feature_nr = _ffi.new("int [1]")
    chip_nm = _ffi.new("char [100]")
    
    chip_nr[0] = 0
    cn = _lib.sensors_get_detected_chips(_ffi.NULL, chip_nr)
    while cn:
        _lib.sensors_snprintf_chip_name(chip_nm, 100, cn)
        chip_name = _ffi.string(chip_nm).decode()
        if target_chip is None or chip_name==target_chip:
            print(chip_name)

            feature_nr[0] = 0
            feature = _lib.sensors_get_features(cn, feature_nr)
            while feature:
                name = _ffi.string(_lib.sensors_get_label(cn, feature)).decode()
                subfeature = None
                if feature.type == _lib.SENSORS_FEATURE_POWER:
                    subfeature = _lib.sensors_get_subfeature(
                        cn, feature, _lib.SENSORS_SUBFEATURE_POWER_INPUT
                    )
                    feature_type = "power"
                    unit = "W"
                elif feature.type == _lib.SENSORS_FEATURE_IN:
                    subfeature = _lib.sensors_get_subfeature(
                        cn, feature, _lib.SENSORS_SUBFEATURE_IN_INPUT
                    )
                    feature_type = "voltage"
                    unit = "V"
                elif feature.type == _lib.SENSORS_FEATURE_CURR:
                    subfeature = _lib.sensors_get_subfeature(
                        cn, feature, _lib.SENSORS_SUBFEATURE_CURR_INPUT
                    )
                    feature_type = "current"
                    unit = "A"
                if subfeature:
                    val = _ffi.new("double [1]")
                    _lib.sensors_get_value(cn, subfeature.number, val)
                    print("%s\t%f %s"%(name, val[0], unit))
                feature = _lib.sensors_get_features(cn, feature_nr)
        cn = _lib.sensors_get_detected_chips(_ffi.NULL, chip_nr)

In [2]:
# print everything
print_sensors()

ina226_vccint_ams-isa-0000
in1	0.000000 V
in2	0.857000 V
power1	0.000000 W
curr1	0.001000 A
ina226_adc_avccaux-isa-0000
in1	0.000000 V
in2	1.826000 V
power1	0.062000 W
curr1	0.031000 A
ina226_mgt1v2-isa-0000
in1	0.000000 V
in2	1.202000 V
power1	0.037000 W
curr1	0.029000 A
ina226_dac_avccaux-isa-0000
in1	0.000000 V
in2	1.817000 V
power1	0.000000 W
curr1	0.000000 A
ina226_vadj_fmc-isa-0000
in1	0.000000 V
in2	0.006000 V
power1	0.000000 W
curr1	0.000000 A
ina226_vcc1v8-isa-0000
in1	0.000000 V
in2	1.806000 V
power1	0.718000 W
curr1	0.398000 A
ina226_vccint-isa-0000
in1	0.000000 V
in2	0.856000 V
power1	0.037000 W
curr1	0.047000 A
ina226_dac_avtt-isa-0000
in1	0.000000 V
in2	3.010000 V
power1	0.037000 W
curr1	0.013000 A
ina226_dac_avcc-isa-0000
in1	0.000000 V
in2	0.927000 V
power1	0.000000 W
curr1	0.003000 A
ina226_mgt1v8-isa-0000
in1	0.000000 V
in2	1.806000 V
power1	0.025000 W
curr1	0.008000 A
ina226_adc_avcc-isa-0000
in1	0.000000 V
in2	0.927000 V
power1	0.162000 W
curr1	0.180000 A
ina226_mgt

In [3]:
# print a specific monitor chip
print_sensors("ina226_dac_avcc-isa-0000")

ina226_dac_avcc-isa-0000
in1	0.000000 V
in2	0.927000 V
power1	0.000000 W
curr1	0.003000 A


In [4]:
from qick import QickSoc
soc = QickSoc()
print_sensors()

resetting clocks: 245.76 491.52
ina226_vccint_ams-isa-0000
in1	0.000000 V
in2	0.858000 V
power1	0.087000 W
curr1	0.089000 A
ina226_adc_avccaux-isa-0000
in1	0.001000 V
in2	1.823000 V
power1	0.412000 W
curr1	0.220000 A
ina226_mgt1v2-isa-0000
in1	0.000000 V
in2	1.202000 V
power1	0.037000 W
curr1	0.037000 A
ina226_dac_avccaux-isa-0000
in1	0.000000 V
in2	1.815000 V
power1	0.312000 W
curr1	0.165000 A
ina226_vadj_fmc-isa-0000
in1	0.000000 V
in2	0.006000 V
power1	0.000000 W
curr1	0.000000 A
ina226_vcc1v8-isa-0000
in1	0.001000 V
in2	1.806000 V
power1	0.968000 W
curr1	0.535000 A
ina226_vccint-isa-0000
in1	0.005000 V
in2	0.857000 V
power1	0.875000 W
curr1	1.033000 A
ina226_dac_avtt-isa-0000
in1	0.000000 V
in2	3.006000 V
power1	0.600000 W
curr1	0.197000 A
ina226_dac_avcc-isa-0000
in1	0.006000 V
in2	0.930000 V
power1	1.212000 W
curr1	1.300000 A
ina226_mgt1v8-isa-0000
in1	0.000000 V
in2	1.805000 V
power1	0.025000 W
curr1	0.008000 A
ina226_adc_avcc-isa-0000
in1	0.001000 V
in2	0.927000 V
power1	0.2250